In [2]:
from theatre_ag import Cast, Episode, SynchronizingClock, TaskQueueActor, default_cost

In [3]:
clock = SynchronizingClock(max_ticks=100)

In [4]:

actor = TaskQueueActor("Alice", clock)

cast = Cast()
cast.add_member(actor)

In [5]:
class Hands(object): 
    def __init__(self):
        self.clean = False
        self.soaped = False

In [6]:
class WashWorkflow(object):

    is_workflow = True

    def __init__(self, washable):
        self.washable = washable

    @default_cost(1)
    def add_soap(self):
        self.washable.soaped = True

    @default_cost(1)
    def rinse(self):
        self.washable.soaped = False

    @default_cost(1)
    def scrub(self):
        if self.washable.soaped:
            self.washable.clean = True

    def wash(self):
        self.add_soap()
        self.scrub()
        self.rinse()

In [7]:
class WashHandsDirection(object):

    def apply(self, cast):
        hands = Hands()
        wash_workflow = WashWorkflow(hands)
        list(cast)[0].allocate_task(wash_workflow.wash)
        list(cast)[0].allocate_task(wash_workflow.wash)
        list(cast)[0].allocate_task(wash_workflow.wash)

In [8]:
direction = WashHandsDirection()
episode = Episode(clock, cast, direction)
episode.perform()

In [9]:
print(actor.task_history)

[wash()[0->3], wash()[3->6], wash()[6->9]]


In [10]:
print(actor.task_history[0].sub_tasks)

[add_soap()[0->1], scrub()[1->2], rinse()[2->3]]


Clock - all activity in a